# Stock Market Analysis Project

**Project Description:**  
In this project, we collected stock market data using the Yahoo Finance public API via the `yfinance` Python library.  
The dataset includes approximately 57,500 daily records for 100 different companies between January 2023 and April 2025.  
We engineered technical and fundamental features such as ATR, RSI, Moving Averages (MA20/MA200), P/E ratio, and Market Capitalization.  
The goal is to perform exploratory data analysis (EDA), feature engineering, and prepare the dataset for future predictive modeling and clustering tasks.




## Data Cleaning & Feature Engineering

In our project, we collected historical stock data for 100 major publicly traded companies using the `yfinance` API. The dataset includes daily price information (Open, High, Low, Close, Volume) from **January 2022 to April 2025** to ensure sufficient historical depth for accurate calculation of technical indicators.

We engineered features such as **RSI (Relative Strength Index), ATR (Average True Range), MA20, and MA200** to capture key signals including volatility, momentum, and both short- and long-term trend directions. Since several indicators require a lookback period (e.g., 200 days for MA200), we deliberately extended our data collection to start a year earlier and then filtered the final dataset to include data from **January 2023** onward, ensuring reliability and stability of computed values.

In addition, we enriched the dataset with **fundamental metrics** such as **P/E ratios** and **Market Capitalization**, mapped to each stock using metadata from Yahoo Finance. We also applied light formatting—rounding numerical values to three decimal places—for improved readability while preserving analytical integrity.

This preprocessing pipeline ensured our data was well-structured, statistically valid, and ready for insightful exploratory analysis.


In [1]:
import yfinance as yf
import pandas as pd

# ------------- STEP 1: Download base stock data -------------
tickers = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'META', 'TSLA', 'NVDA', 'BRK-B', 'V', 'JPM',
    'JNJ', 'UNH', 'XOM', 'PG', 'MA', 'HD', 'CVX', 'ABBV', 'LLY', 'MRK',
    'AVGO', 'PEP', 'KO', 'COST', 'PFE', 'ADBE', 'TMO', 'WMT', 'CSCO', 'BAC',
    'ORCL', 'MCD', 'AMD', 'ABT', 'CRM', 'DIS', 'NFLX', 'NKE', 'INTC', 'VZ',
    'LIN', 'WFC', 'ACN', 'DHR', 'TXN', 'QCOM', 'AMGN', 'MDT', 'NEE', 'PM',
    'UPS', 'BMY', 'MS', 'RTX', 'UNP', 'LOW', 'SPGI', 'PLD', 'IBM', 'GS',
    'INTU', 'SBUX', 'ISRG', 'AXP', 'T', 'CAT', 'DE', 'BLK', 'GE', 'SYK',
    'NOW', 'AMAT', 'ELV', 'CI', 'ZTS', 'CB', 'SCHW', 'ADI', 'MDLZ', 'MMC',
    'LRCX', 'GILD', 'MO', 'PGR', 'BKNG', 'ADP', 'LMT', 'TGT', 'BA', 'USB',
    'SO', 'C', 'VRTX', 'GM', 'BDX', 'FDX', 'MU', 'CL', 'APD', 'TMUS'
]

print("Downloading historical stock data...")
data = yf.download(
    tickers=tickers,
    start="2022-01-01",
    end="2025-04-21",
    interval="1d",
    group_by='ticker',
    auto_adjust=True,
    threads=True
)

# ------------- STEP 2: Organize and concatenate data -------------
dfs = []
for ticker in tickers:
    df = data[ticker].copy()
    df['Ticker'] = ticker
    dfs.append(df)

final_df = pd.concat(dfs)
final_df.reset_index(inplace=True)

print("Data has been structured successfully.")

# ------------- STEP 3: Add technical indicators -------------

# ATR (Average True Range)
high_low = final_df['High'] - final_df['Low']
high_close = (final_df['High'] - final_df['Close'].shift()).abs()
low_close = (final_df['Low'] - final_df['Close'].shift()).abs()
ranges = pd.concat([high_low, high_close, low_close], axis=1)
true_range = ranges.max(axis=1)
final_df['ATR_14'] = true_range.ewm(span=14, adjust=False).mean()

# RSI (Relative Strength Index)
delta = final_df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.ewm(span=14, adjust=False).mean()
avg_loss = loss.ewm(span=14, adjust=False).mean()
rs = avg_gain / avg_loss
final_df['RSI_14'] = 100 - (100 / (1 + rs))

# Replace values for initial days with insufficient data
final_df.loc[final_df.index < 14, 'ATR_14'] = None
final_df.loc[final_df.index < 14, 'RSI_14'] = None

print("Technical indicators (ATR, RSI) added.")

# ------------- STEP 4: Add fundamental metrics (P/E, Market Cap) -------------

print("Fetching fundamental data (P/E ratio, Market Cap)...")

pe_ratios = {}
market_caps = {}

for ticker in tickers:
    stock = yf.Ticker(ticker)
    try:
        pe_ratios[ticker] = stock.info.get('trailingPE', None)
        market_caps[ticker] = stock.info.get('marketCap', None)
    except:
        pe_ratios[ticker] = None
        market_caps[ticker] = None

final_df['P/E'] = final_df['Ticker'].map(pe_ratios)
final_df['Market_Cap'] = final_df['Ticker'].map(market_caps)

# Calculate Moving Averages
final_df['MA20'] = final_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
final_df['MA200'] = final_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=200).mean())


print("Fundamental indicators added.")

# ------------- STEP 5: Save data to CSV -------------
final_df = final_df[final_df['Date'] >= '2023-01-01']
final_df = final_df.round(3)
final_df.to_csv('stocks_data_full.csv', index=False)
print("Data successfully saved to 'stocks_data_full.csv' with all indicators included.")

[*********************100%***********************]  100 of 100 completed


Data has been structured successfully.
Technical indicators (ATR, RSI) added.
Fetching fundamental data (P/E ratio, Market Cap)...
Fundamental indicators added.
Data successfully saved to 'stocks_data_full.csv' with all indicators included.


In [ ]:
# ------------- STEP 6: Add Sector column to existing CSV -------------

print("Loading existing CSV and adding Sector column...")

# טען את הקובץ הקיים
df = pd.read_csv('stocks_data_full.csv')
sectors = {
    'AAPL': 'Technology',
    'MSFT': 'Technology',
    'GOOG': 'Communication Services',
    'AMZN': 'Consumer Discretionary',
    'META': 'Communication Services',
    'TSLA': 'Consumer Discretionary',
    'NVDA': 'Technology',
    'BRK-B': 'Financials',
    'V': 'Technology',
    'JPM': 'Financials',
    'JNJ': 'Health Care',
    'UNH': 'Health Care',
    'XOM': 'Energy',
    'PG': 'Consumer Staples',
    'MA': 'Technology',
    'HD': 'Consumer Discretionary',
    'CVX': 'Energy',
    'ABBV': 'Health Care',
    'LLY': 'Health Care',
    'MRK': 'Health Care',
    'AVGO': 'Technology',
    'PEP': 'Consumer Staples',
    'KO': 'Consumer Staples',
    'COST': 'Consumer Staples',
    'PFE': 'Health Care',
    'ADBE': 'Technology',
    'TMO': 'Health Care',
    'WMT': 'Consumer Staples',
    'CSCO': 'Technology',
    'BAC': 'Financials',
    'ORCL': 'Technology',
    'MCD': 'Consumer Discretionary',
    'AMD': 'Technology',
    'ABT': 'Health Care',
    'CRM': 'Technology',
    'DIS': 'Communication Services',
    'NFLX': 'Communication Services',
    'NKE': 'Consumer Discretionary',
    'INTC': 'Technology',
    'VZ': 'Communication Services',
    'LIN': 'Materials',
    'WFC': 'Financials',
    'ACN': 'Technology',
    'DHR': 'Health Care',
    'TXN': 'Technology',
    'QCOM': 'Technology',
    'AMGN': 'Health Care',
    'MDT': 'Health Care',
    'NEE': 'Utilities',
    'PM': 'Consumer Staples',
    'UPS': 'Industrials',
    'BMY': 'Health Care',
    'MS': 'Financials',
    'RTX': 'Industrials',
    'UNP': 'Industrials',
    'LOW': 'Consumer Discretionary',
    'SPGI': 'Financials',
    'PLD': 'Real Estate',
    'IBM': 'Technology',
    'GS': 'Financials',
    'INTU': 'Technology',
    'SBUX': 'Consumer Discretionary',
    'ISRG': 'Health Care',
    'AXP': 'Financials',
    'T': 'Communication Services',
    'CAT': 'Industrials',
    'DE': 'Industrials',
    'BLK': 'Financials',
    'GE': 'Industrials',
    'SYK': 'Health Care',
    'NOW': 'Technology',
    'AMAT': 'Technology',
    'ELV': 'Health Care',
    'CI': 'Health Care',
    'ZTS': 'Health Care',
    'CB': 'Financials',
    'SCHW': 'Financials',
    'ADI': 'Technology',
    'MDLZ': 'Consumer Staples',
    'MMC': 'Financials',
    'LRCX': 'Technology',
    'GILD': 'Health Care',
    'MO': 'Consumer Staples',
    'PGR': 'Financials',
    'BKNG': 'Consumer Discretionary',
    'ADP': 'Industrials',
    'LMT': 'Industrials',
    'TGT': 'Consumer Staples',
    'BA': 'Industrials',
    'USB': 'Financials',
    'SO': 'Utilities',
    'C': 'Financials',
    'VRTX': 'Health Care',
    'GM': 'Consumer Discretionary',
    'BDX': 'Health Care',
    'FDX': 'Industrials',
    'MU': 'Technology',
    'CL': 'Consumer Staples',
    'APD': 'Materials',
    'TMUS': 'Communication Services'
}



df['Sector'] = df['Ticker'].map(sectors)
df.to_csv('stocks_data_full.csv', index=False)
print("Sector column added successfully to 'stocks_data_full.csv'.")

Loading existing CSV and adding Sector column...
Sector column added successfully to 'stocks_data_full.csv'.
